In [ ]:
#сделали поддержку большого датасета
#
#здесь мы выгружаем GPT-2 и пытаемся дообучать на кроссдоменном датасете
import torch
from transformers import GPT2Tokenizer, PreTrainedTokenizer
import codecs
import numpy as np
import os, pickle, random
import psutil
import warnings
from typing import Dict, List, Optional
from torch.utils.data import DataLoader
from collections import deque

from trl import AutoModelForCausalLMWithValueHead
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import Trainer, TrainingArguments



import result_gpt_res

E:\ProgramData\Anaconda3\envs\tf\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
E:\ProgramData\Anaconda3\envs\tf\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
E:\ProgramData\Anaconda3\envs\tf\lib\site-packages\numpy\.libs\libopenblas.JPIJNSWNNAN3CE6LLI5FWSPHUT2VXMTH.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"
E:\ProgramData\Anaconda3\envs\tf\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [ ]:
#!pip install transformers
#!pip install trl
#!pip install accelerate -U
#!pip install transformers[torch]

In [ ]:
torch.cuda.get_device_name(0)

In [ ]:
#Модель, с которой стартовал я: "gpt2-medium"
#Модель после обучения у меня называется: "gpt2_finetuned_3"
#Можно использовать более легковесную модель "gpt2"

#model_name = "gpt2-medium"
model_name = "gpt2_finetuned_3"

In [ ]:
model_name2save = "gpt2_finetuned_3"

In [ ]:
# 1. load a pretrained model

tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
#специальные токены
T_OUT = tokenizer.encode('<OUT>')[0]
T_END = tokenizer.encode('<END>')[0]
T_PAD = tokenizer.encode('|PAD|')[0]
T_124 = tokenizer.encode('<124>')[0]

In [ ]:
#Здесь добавляем токены ревордов
reward_to_substr = {}
rmax = 25
rmin = -25
step = 0.1
for r in np.arange(rmin, rmax, step):
    r = np.round(r, 2)
    str_r = f'<r{r}>'
    reward_to_substr[r] = str_r
substr_to_reward = {v: k for k, v in reward_to_substr.items()}
print('substr_to_reward', substr_to_reward)
print('reward_to_substr', reward_to_substr)

In [ ]:
#Учим эту модель.
#Модификация токенайзера 
#добавляем несколько токенов нашей разметки

#сюда надо фигануть весь словарь картинок
video_tokens_cnt = 1024
video_tokens = []
for i in range(video_tokens_cnt):
    video_tokens.append(f'<{i}>')
    
#и токены ревордов тоже добавить
    
special_tokens_dict = {'additional_special_tokens': video_tokens + ['<IN>','<OUT>','<END>','|PAD|'] + list(reward_to_substr.values())}
num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)
#print(tokenizer.all_special_tokens)
tokenizer.pad_token = '|PAD|'

In [ ]:
r_max_token = tokenizer.encode(list(reward_to_substr.values())[-1])[0]
r_min_token = tokenizer.encode(list(reward_to_substr.values())[0])[0]
out_token = tokenizer.encode('<OUT>')[0]

In [ ]:
#r_mapping {1231:0.1, 1547:1.2} и так далее.
#маппинг с номеров токенов на реворды
r_mapping = {}
for substr in list(reward_to_substr.values()):
    r_token = tokenizer.encode(substr)[0]
    r_mapping[r_token] = substr_to_reward[substr]
print(r_mapping)

In [ ]:
print('mem', result_gpt_res.getmem())
model = result_gpt_res.AutoModelForCausalLMWithValueHeadCL.from_pretrained(model_name)
#r_mapping {1231:0.1, 1547:1.2} и так далее.
#маппинг с номеров токенов на реворды
model.r_mapping = r_mapping
#токен <OUT>, с него начинается прогноз
model.out_token = out_token

model._modules['pretrained_model'].resize_token_embeddings(len(tokenizer))

In [ ]:
from transformers import TextDataset,DataCollatorForLanguageModeling#,LineByLineTextDataset
#Загрузить датасет
print('mem', result_gpt_res.getmem())

batch_size = 2
block_size = 150
#block_size -  размер контекста,который мы учитываем при обучении. 
#Он меньше, чем максимально доступный трансформеру.
#Чем меньше, тем меньше расход GPU на обучении

train_dataset = result_gpt_res.LineByLineTextDataset(
          tokenizer=tokenizer,
          #file_path='data/all_txt_cut_r.txt',
          file_path='data/all_txt_cut_r_low.txt',#ЭТО МЕЛКИЙ ДАТАСЕТ, ЧИСТО ДЛЯ ПРОВЕРКИ ПРЦЕДУРЫ ОБУЧЕНИЯ
          block_size=block_size)
print("data loaded")
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [ ]:
print('mem', result_gpt_res.getmem())
# если не лезет в память, можно уменьшать **per_device_train_batch_size**
# gradient_accumulation_steps лучше повышать со временем. То есть вначале вы обучаете модель на датасете, имея
#gradient_accumulation_steps = 2, так обучение идёт быстрее всего. Затем утыкаетесь в оптимум, стопаете, сохраняетесь (дальше)
#удваиваете gradient_accumulation_steps

model.tokenizer = tokenizer
training_args = TrainingArguments(
    output_dir="./models/gpt_bot", #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=80000, # number of training epochs
    per_device_train_batch_size= batch_size, # batch size for training
    gradient_accumulation_steps=2,
    #per_device_eval_batch_size=50,  # batch size for evaluation
    eval_steps = 3, # Number of update steps between two evaluations.
    save_steps=1000, # after # steps model is saved 
    warmup_steps=200,# number of warmup steps for learning rate scheduler
    prediction_loss_only=True,
    )

trainer = Trainer(#result_gpt_res.TrainerUnsampled(
    model=model,
    args=training_args,
    data_collator=data_collator,
    #train_dataset=tokenized_dataset['train'],
    train_dataset=train_dataset,
    #eval_dataset=test_dataset,
    #compute_metrics=compute_metrics,
)

torch.cuda.empty_cache()

In [ ]:
os.environ["TOKENIZERS_PARALLELISM"] = "false"
warnings.filterwarnings("ignore")

In [ ]:
for i in range(10000):
    try:
        trainer.train()
    except Exception as e:
        print(e)
        # save it locally
        model.save_pretrained(model_name2save)
        tokenizer.save_pretrained(model_name2save)

In [ ]:
ones = torch.ones([122, 122, 1024], device=model.v_head[0].weight.device)
print('ones', model.v_head(ones)[0,0,0:3])
#0.2236

In [ ]:
#засейвить модель
if 1:
    # save it locally
    model.save_pretrained(model_name2save)
    tokenizer.save_pretrained(model_name2save)
1

In [ ]:
#проверить, что модель вообще генерит

In [ ]:
def answer(text):
    #text = '\n' + text
    inpt = tokenizer.encode(text, return_tensors="pt")
    inpt= inpt.cuda()
    print(text)

    #out = model.generate(inpt,  max_length=len(inpt[0])+300, do_sample=True, top_k=5, top_p=0.95, temperature=1, eos_token_id=T_END, pad_token_id=T_PAD)
    out = model.generate(inpt,  max_length=len(inpt[0])+300, do_sample=False, top_k=1, top_p=0.95, temperature=0.0001, eos_token_id=T_END, pad_token_id=T_PAD)

    out_tokens = torch.where(out[0]==T_OUT)
    #last_repl = out[0][out_tokens[0][-1]+1:-1]
    #print(out)
    last_repl = out[0][out_tokens[0][-1]+1:-1]
    repl = tokenizer.decode(last_repl)

    print(repl)

In [ ]:
#a11
text = '<916><61><922><922><922><922><922><922><55><906><1022><830><376><830><830><830><830><61><771><402><438><751><830><925><925><830><61><580><519><920><462><830><925><925><925><61><120><519><255><327><61><925><328><55><483><657><255><49><328><830><925><147><120><287><328><830><328><61><830><706><957><29><222><925><55><55><55><328><851><590><596><164><376><61><328><328><328><801><925><422><255> description:'
text = f"<IN>{text}<OUT>"
answer(text)
text = '<916><61><922><922><922><922><922><922><55><906><1022><830><376><830><830><830><830><61><771><402><438><751><830><925><925><830><61><580><519><920><462><830><925><925><925><61><120><519><255><327><61><925><328><55><483><657><255><49><328><830><925><147><120><287><328><830><328><61><830><706><957><29><222><925><55><55><55><328><851><590><596><164><376><61><328><328><328><801><925><422><255> save hp plan:'
text = f"<IN>{text}<OUT>"
answer(text)

In [ ]:
text = '<705><993><941><325><418><751><937><666><681><110><418><623><72><577><261><358><313><325><974><701><884><414><564><414><63><450><624><176><806><894><564><966><626><957><638><749><66><893><924><634><595><143><120><433><202><556><328><910><57><63><692><189><294><414><751><812><49><909><125><797><261><681><503><552><761><382><657><92><343><671><473><761><505><894><857><788><499><937><334><459><746> description:'
text = f"<IN>{text}<OUT>"
answer(text)


In [ ]:
text = '<32><794><685><585><276><610><578><414><769><129><450><681><966><813><250><256><862><749><438><639><551><54><794><887><500><681><520><996><957><202><848><110><985><164><54><807><520><382><508><473><725><358><110><325><597><54><937><49><473><684><156><481><417><498><718><746><404><455><745><725><363><666><325><560><176><848><438><140><172><848><473><714><872><737><120><477><136><813><893><120><746> description:'
text = f"<IN>{text}<OUT>"
answer(text)

